## Imports

In [31]:
#%pip install pandas
#%pip install numpy
#%pip install sklearn

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.calibration import CalibratedClassifierCV

# Just for visuals
pd.set_option('display.max_colwidth', None)

# 1. Dataset Preparation

## Load & Augment Pre-processed Dataset

In [33]:
df_o = pd.read_pickle('oyez_proccessed.pkl')

print(f'There are {len(df_o)} cases.')

There are 2380 cases.


In [34]:
# Create a mirrored case for each case, where the parties are swapped to prevent favoring first_party
def mirror_case(row):
    return {
        'ID': f"{row['ID']}_mirrored",
        'name': f"{row['name']} (Mirrored)",
        'first_party': row['second_party'],
        'second_party': row['first_party'],
        #'winning_party': row['second_party'] if row['winner_index'] == 0 else row['first_party'],
        'winning_party': row['winning_party'],
        #future imporvement: further augment facts/question/conclusion
        'facts': row['facts'],
        'question': row['question'],
        'conclusion': row['conclusion'], 
        'winner_index': 1 - row['winner_index']
}
mirrored = []
for index, row in df_o.iterrows():
    mirrored.append(mirror_case(row))
df_m = pd.DataFrame(mirrored)

df = pd.concat([df_o, df_m])

In [35]:
print(f'There are {len(df)} cases.')
print(f'There are {len(df[df["winner_index"]==0])} rows for class 0.')
print(f'There are {len(df[df["winner_index"]==1])} rows for class 1.')

There are 4760 cases.
There are 2380 rows for class 0.
There are 2380 rows for class 1.


In [36]:
display(df.sample(n=5))

,ID,name,first_party,second_party,winning_party,facts,question,conclusion,winner_index
2141,62831,Ayestas v. Davis,Carlos Manuel Ayestas,"Lorie Davis, Director, Texas Department of Criminal Justice (Institutional Division)",Ayestas,"In July 1997, Carlos Manuel Ayestas, a Honduran national, was convicted and sentenced to death for the murder of Santiaga Paneque at her home in Houston, Texas. Ayestas appealed his conviction and filed for state habeas relief based on a claim of ineffective assistance of counsel; he argued that his trial counsel failed to secure the testimony of family members who could have presented mitigating evidence. The state habeas court found that Ayestas’ trial attorney was not ineffective and denied his application for habeas relief. The Texas Court of Criminal Appeals adopted the findings of the state habeas court and denied relief as well. In 2009, Ayestas, with a new attorney, filed a federal habeas petition and claimed that his trial counsel had been ineffective because he had failed to conduct a reasonable investigation that would have uncovered “available and abundant” mitigating factors. However, the district court found that his claim was procedurally defaulted because it had not been raised in the state habeas proceeding and denied relief. The US Court of Appeals for the Fifth Circuit affirmed. The US Supreme Court subsequently decided Martinez v. Ryan, which held that ineffectiveness of state habeas counsel in failing to bring an ineffective assistance of trial counsel claim may excuse a procedural default. The Supreme Court later extended the holding of Martinez to Texas’ habeas scheme in Trevino v. Thaler. In light of these decisions, Ayestas asked for a rehearing. On rehearing in the district court, Ayestas filed a motion for investigative assistance that was “reasonably necessary” to develop his broader ineffective assistance of counsel claim under 18 YOU.S.C. § 3599(f). The district court denied the motion based on Fifth Circuit precedent that holds that investigative assistance is not “reasonably necessary” unless the habeas petitioner can carry the ultimate burden of proof on the underlying claim at the time of the request for investigative assistance. The district court determined that Ayestas could not meet his burden of proof on the ineffective assistance of counsel claim and therefore denied the motion. The appellate court affirmed.",Are investigative services under 18 YOU.S.C. § 3599(f) only “reasonably necessary” when the habeas petitioner requesting the services can meet the burden of proof for the underlying claim for habeas relief at the time of the request for the investigative services?,"In a unanimous opinion authored by Justice Alito, after explaining that it had jurisdiction to hear the appeal because it related to a judicial rather than an administrative decision, the Court held that the Fifth Circuit applied the wrong legal standard in affirming the denial of Ayestas' claim for the investigative funding needed to prove his entitlement to federal habeas relief under 18 USC § 3599(f). Analyzing the statutory language, the Court concluded that it called for district courts to exercise their discretion in determining whether a reasonable attorney would deem the services sought by the funding to be sufficiently important to the case in order to meet the ""reasonably necessary"" requirement. The Fifth Circuit's ""substantial need"" standard was more stringent, and therefore improper. The Court explained that the Fifth Circuit also erred in requiring Ayestas to present ""a viable constitutional claim that [was] not procedurally barred,"" as this rule had become too restrictive in light of Trevino, and lower courts could potentially err in denying funding requests in cases where such funding could allow a petitioner to overcome a procedural default. On the other hand, the Court also explained that district courts had broad discretion in ruling on § 3599(f) funding requests, and w

In [37]:
# Facts character stats
df['facts'].apply(lambda x: len(str(x))).describe()

count    4760.000000
mean     1128.322689
std       530.495783
min        98.000000
25%       773.000000
50%      1062.000000
75%      1395.500000
max      6113.000000
Name: facts, dtype: float64

In [38]:
# Facts word stats
df['facts'].apply(lambda x: len(str(x).split())).describe()

count    4760.000000
mean      176.907983
std        84.457133
min        14.000000
25%       121.000000
50%       166.000000
75%       221.000000
max       974.000000
Name: facts, dtype: float64

# 2. AutoJudge
Given the neutral absolute facts, predict the decision of the case.